In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.filters import Filters as filters
from ptrail.features.kinematic_features import KinematicFeatures
from ptrail.features.temporal_features import TemporalFeatures
from ptrail.preprocessing.interpolation import Interpolation
from ptrail.utilities.conversions import Conversions as con
import ptrail.utilities.constants as const
import datetime as dt

# Reading the gulls dataset and converting to PTRAILDataFrame.
# Also, lets, print the first 5 points of the dataset to
# see how the dataframe looks.
gulls = pd.read_csv('./data/gulls.csv')
gulls = PTRAILDataFrame(gulls,
                        latitude='location-lat',
                        longitude='location-long',
                        datetime='timestamp',
                        traj_id='tag-local-identifier',
                        rest_of_columns=[])

In [2]:
pdf = pd.read_csv('./data/geolife_sample.csv')
geolife = PTRAILDataFrame(data_set=pdf,
                             latitude='lat',
                             longitude='lon',
                             datetime='datetime',
                             traj_id='id')


In [3]:
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = PTRAILDataFrame(atlantic,
                          latitude='Latitude',
                          longitude='Longitude',
                          datetime='DateTime',
                          traj_id='ID',
                          rest_of_columns=[])
atlantic = TemporalFeatures.create_date_column(atlantic)

In [4]:
pdf = pd.read_csv('./data/starkey.csv')
starkey = PTRAILDataFrame(data_set=pdf,
                          latitude='lat',
                          longitude='lon',
                          datetime='DateTime',
                          traj_id='Id')

In [5]:
t1 = gulls.reset_index().loc[gulls.reset_index()[const.TRAJECTORY_ID] == '91732'][[const.TRAJECTORY_ID, const.DateTime, const.LAT, const.LONG]]
t1 = PTRAILDataFrame(data_set=t1,
                     datetime='DateTime',
                     traj_id='traj_id',
                     latitude='lat',
                     longitude='lon')

In [6]:
def segment_traj_by_week(df):
    """
        Given a dataframe containing trajectory data, segment all
        the trajectories by each week.

        Parameters
        ----------
            df: PTRAILDataFrame
                The dataframe containing trajectory data.

        Returns
        -------
            pandas.core.dataframe.DataFrame:
                The dataframe containing segmented trajectories
                with a new column added called segment_id
    """
    # First, create the date column and get all the unique traj_ids
    # in the dataframe..
    df = TemporalFeatures.create_date_column(df)
    ids_ = list(df.traj_id.value_counts().keys())

    # Get the ideal number of IDs by which the dataframe is to be split.
    df_chunks = []
    for i in range(len(ids_)):
        small_df = df.reset_index().loc[df.reset_index()[const.TRAJECTORY_ID] == ids_[i]]
        df_chunks.append(small_df)

    # Now, iterate over the entire dataframe and then segment
    # the trajectories by 1 week each.
    results = []
    for i in range(len(ids_)):
        # Take the traj_df of a single Trajectory out from the
        # list of chunks and find their max and min timestamps.
        traj = df_chunks[i]
        t_max = traj.reset_index()[const.DateTime].max()
        t_min = traj.reset_index()[const.DateTime].min()

        # For iteration purposes, set t_1 to min and t_2 to
        # t_1 + 7 days.
        t_1 = t_min
        t_2 = t_1 + dt.timedelta(days=7)
        seg_id = 1

        # Now, segment the trajectories into smaller segments
        # wherein each segment contains the points of a span
        # of 7 days only.
        while t_2 < t_max:
            if t_2 < t_max:
                seg = filters.filter_by_date(traj,
                                             start_date=t_1.strftime('%Y-%m-%d'),
                                             end_date=t_2.strftime('%Y-%m-%d'))
                # Once filtered, assign the segment with a segment ID.
                seg['seg_id'] = seg_id

                # Increment the segment id, t_1 and t_2 values by
                # 1, 7 days and 7 days respectively to continue
                # the iteration.
                seg_id += 1
                t_1 += dt.timedelta(days=7)
                t_2 += dt.timedelta(days=7)
                results.append(seg.drop(columns=['index', 'level_0']))

            # If, t_2 is greater than the max time present in the
            # trajectory, then assign t_2 = max and proceed
            # further with segmentation.
            elif t_2 >= t_max:
                seg = filters.filter_by_date(traj,
                                             start_date=t_1.strftime('%Y-%m-%d'),
                                             end_date=t_max.strftime('%Y-%m-%d'))
                # Once filtered, assign the segment with a segment ID.
                seg['seg_id'] = seg_id

                # Increment the segment id, t_1 and t_2 values by
                # 1, 7 days and 7 days respectively to continue
                # the iteration.
                seg_id += 1
                t_1 += dt.timedelta(days=7)
                t_2 += dt.timedelta(days=7)
                results.append(seg.drop(columns=['index', 'level_0']))

    # Finally, concat the dataframes, set the index as
    # [traj_id, seg_id, DateTime].
    return pd.concat(results).reset_index().set_index(['traj_id', 'seg_id', 'DateTime'])

In [7]:
%%time

seg = segment_traj_by_week(starkey)

CPU times: user 3min 49s, sys: 408 ms, total: 3min 49s
Wall time: 3min 49s


In [8]:
print(len(starkey))
seg

287136


lat         lon  StarkeyTime  \
traj_id   seg_id DateTime                                                  
910313E37 1      1993-05-06 17:04:47  45.218440 -118.566257    168829487   
                 1993-05-06 17:21:12  45.218985 -118.565890    168826332   
                 1993-05-06 17:27:53  45.219536 -118.565141    168830873   
                 1993-05-06 17:51:48  45.219260 -118.565516    168832308   
                 1993-05-06 18:40:04  45.220128 -118.561336    168835204   
...                                         ...         ...          ...   
950227D02 1      1996-08-11 07:18:14  45.223289 -118.567155    271869494   
                 1996-08-11 09:13:06  45.223008 -118.567911    271876386   
                 1996-08-11 11:06:48  45.223003 -118.568293    271883208   
                 1996-08-11 13:06:05  45.224342 -118.569095    271890365   
                 1996-08-11 15:03:44  45.225173 -118.567589    271897424   

                                        GMDate    GMTime   LocDate   LocTime  \
traj_id   seg_id DateTime                                                      
910313E37 1      1993-05-06 17:04:47  01:04:47  19930507  19930506  17:04:47   
                 1993-05-06 17:21:12  01:21:12  19930507  19930506  17:21:12   
                 1993-05-06 17:27:53  01:27:53  19930507  19930506  17:27:53   
                 1993-05-06 17:51:48  01:51:48  19930507  19930506  17:51:48   
                 1993-05-06 18:40:04  02:40:04  19930507  19930506  18:40:04   
...                                        ...       ...       ...       ...   
950227D02 1      1996-08-11 07:18:14  15:18:14  19960811  19960811  07:18:14   
                 1996-08-11 09:13:06  17:13:06  19960811  19960811  09:13:06   
                 1996-08-11 11:06:48  19:06:48  19960811  19960811  11:06:48   
                 1996-08-11 13:06:05  21:06:05  19960811  19960811  13:06:05   
                 1996-08-11 15:03:44  23:03:44  19960811  19960811  15:03:44   

                                      RadNum Species    UTME     UTMN  Year  \
traj_id   seg_id DateTime                                                     
910313E37 1      1993-05-06 17:04:47      67       E  377026  5008419    93   
                 1993-05-06 17:21:12      67       E  377053  5008474    93   
                 1993-05-06 17:27:53      67       E  377108  5008516    93   
                 1993-05-06 17:51:48      67       E  377083  5008494    93   
                 1993-05-06 18:40:04      67       E  377406  5008591    93   
...                                      ...     ...     ...      ...   ...   
950227D02 1      1996-08-11 07:18:14     417       D  376978  5008937    96   
                 1996-08-11 09:13:06     417       D  376902  5008934    96   
                 1996-08-11 11:06:48     417       D  376865  5008927    96   
                 1996-08-11 13:06:05     417       D  376806  5009065    96   
                 1996-08-11 15:03:44     417       D  376927  5009150    96   

                                      Grensunr  Grensuns  Obswt        Date  
traj_id   seg_id DateTime                                                    
910313E37 1      1993-05-06 17:04:47  12:34:00  03:10:00   1.67  1993-05-06  
                 1993-05-06 17:21:12  12:34:00  03:10:00   1.73  1993-05-06  
                 1993-05-06 17:27:53  12:34:00  03:10:00   1.73  1993-05-06  
                 1993-05-06 17:51:48  12:34:00  03:10:00   1.73  1993-05-06  
                 1993-05-06 18:40:04  12:34:00  03:10:00   1.46  1993-05-06  
...                                        ...       ...    ...         ...  
950227D02 1      1996-08-11 07:18:14  12:51:00  03:10:00   1.43  1996-08-11  
                 1996-08-11 09:13:06  12:51:00  03:10:00   1.43  1996-08-11  
                 1996-08-11 11:06:48  12:51:00  03:10:00   1.43  1996-08-11  
                 1996-08-11 13:06:05  12:51:00  03:10:00   1.56  1996-08-11  
                 1996-08-11 15:03:44  12:51:00  03

In [9]:
seg.reset_index().loc[seg.reset_index()['seg_id'] == 3 & seg.reset_index()['traj_id'] == '1']

TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]

In [ ]:
# stats = KinematicFeatures.generate_kinematic_stats(gulls)

In [ ]:
# stats.head(16)